In [ ]:
# use env napari
import numpy as np
import pandas as pd
import seaborn as sns

import time, os, sys
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import matplotlib as mpl
import cv2
import glob
import tifffile

from PIL import Image

from scroutines.config_plots import *

# import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'
# from cellpose import utils, io, plot, models

In [ ]:
# imgs_files = np.sort(glob.glob(
#     f"/u/home/f/f7xiesnm/project-zipursky/data/CTb_tracing/{sample}_20x_{fov}.tif"
# ))
# print(imgs_files)

In [ ]:
files = np.sort(glob.glob(
    "/u/home/f/f7xiesnm/project-zipursky/data/CTb_tracing/MAX_ctb*_centroids.npy"
))
np.unique([os.path.basename(f)[:9] for f in files])

In [ ]:
meta = [
    ('MAX_ctb02', 'DR_adult', 'RL', 'LM', 'AL', 'DAPI'),
    ('MAX_ctb03', 'NR_adult', 'RL', 'AL', 'LM', 'DAPI'),
    ('MAX_ctb07', 'DR_P34',   'LM', 'AL', 'RL', 'DAPI'),
    ('MAX_ctb08', 'DR_P34_v2',   'LM', 'AL', 'RL', 'DAPI'),
    ('MAX_ctb10', 'NR_P14',   'LM', 'RL', 'DAPI'),
]

In [ ]:
anno2samp = {item[1]: item[0] for item in meta}
anno2samp


# one FOV

In [ ]:
sample = 'MAX_ctb03'
fov = 'slice1_fov1'
labels = ['RL', 'AL', 'LM', 'DAPI'] 

img_file = f"/u/home/f/f7xiesnm/project-zipursky/data/CTb_tracing/{sample}_20x_{fov}.tif"
print(img_file)

a0, a1, a2, a3 = tifffile.imread(img_file)
a0 = np.array(Image.fromarray(a0).resize((512,512)))
a1 = np.array(Image.fromarray(a1).resize((512,512)))
a2 = np.array(Image.fromarray(a2).resize((512,512)))
a3 = np.array(Image.fromarray(a3).resize((512,512)))
imgs = [a0,a1,a2,a3]

xs = []
ys = []
for i in range(4):
    coord_file = f"/u/home/f/f7xiesnm/project-zipursky/data/CTb_tracing/{sample}_20x_{fov}_c{i}_centroids.npy"
    dat = np.load(coord_file, allow_pickle=True).item()
    x = dat['centroid-0'] # vertical along the image - 
    y = dat['centroid-1'] # horizontal along the image - higher means closer to pia
    xs.append(x)
    ys.append(y)



In [ ]:
def plot_oneset(imgs, xs, ys, labels, show_img=True, show_xy=True):
    """
    """
    fig, axs = plt.subplots(1,4,figsize=(4*6,1*5))
    for i in range(4): 
        ai = imgs[i]
        x = xs[i]
        y = ys[i]
        lbl = labels[i]
        ax = axs.flat[i]

        if show_img:
            vmax = np.percentile(ai, 99)
            vmin = np.percentile(ai, 1)
            ax.imshow(ai, vmax=vmax, vmin=vmin, cmap='viridis')

        if show_xy:
            ax.scatter(y, x, facecolors='none', edgecolors='red')
            if not show_img:
                # ax.invert_yaxis()
                ax.set_xlim([0, 512])
                ax.set_ylim([512, 0])

        ax.grid(False)
        ax.set_aspect('equal')
        ax.set_title(lbl)

In [ ]:
plot_oneset(imgs, xs, ys, labels, show_img=True,  show_xy=False)
plot_oneset(imgs, xs, ys, labels, show_img=False, show_xy=True)
plot_oneset(imgs, xs, ys, labels, show_img=True,  show_xy=True)

In [ ]:
bins_fine = np.linspace(0, 512, num=64)
label_colors = {
    'LM': 'C0',
    'RL': 'C1',
    'AL': 'C2',
    'DAPI': 'gray',
}

fig, ax = plt.subplots()
for i in range(4):
    depth = 512-ys[i]
    lbl = labels[i]
    clr = label_colors[lbl]
    sns.histplot(depth, bins=bins_fine, stat='percent', cumulative=True, element='step', linewidth=2, fill=False, label=lbl, color=clr, ax=ax)
ax.legend()
ax.grid(False)
sns.despine(ax=ax)
ax.set_ylabel('Cumu. percent')
ax.set_xlabel('Pia -> vent')
plt.show()

fig, ax = plt.subplots()
for i in range(4):
    depth = 512-ys[i]
    lbl = labels[i]
    clr = label_colors[lbl]
    sns.histplot(depth, bins=bins_fine, stat='percent', cumulative=False, element='step', linewidth=2, fill=False, label=lbl, color=clr, ax=ax)
ax.legend()
ax.grid(False)
sns.despine(ax=ax)
ax.set_ylabel('Percent')
ax.set_xlabel('Pia -> vent')
plt.show()

# All FOVs

In [ ]:
res = {}
for meta_samp in meta:
    sample, anno, labels = meta_samp[0], meta_samp[1], meta_samp[2:]
    print(sample, anno, labels)
    res[anno] = {}
    
    for i, label in enumerate(labels):
        files = np.sort(glob.glob(
            f"/u/home/f/f7xiesnm/project-zipursky/data/CTb_tracing/{sample}_*_c{i}_centroids.npy"
        ))
        # print(files[:5], len(files))

        ys = []
        for file in files:
            dat = np.load(file, allow_pickle=True).item()
            x = dat['centroid-0'] # vertical along the image - 
            y = dat['centroid-1'] # horizontal along the image - higher means closer to pia
            a = dat['area']
            # print(y.shape)
            ys.append(y)
        depth = 512 - np.hstack(ys)
        print(len(depth))
        res[anno][label] = depth

    #     break
    # break

In [ ]:
def plot(sample_anno, projs, bins_fine, bins_crse, title=None):
    """
    """
    fig, axs = plt.subplots(2,2,figsize=(2*6,2*5))
    if title is None:
        fig.suptitle(sample_anno)
    else:
        fig.suptitle(title)
    ax = axs.flat[0]
    for proj in projs:
        depth = res[sample_anno][proj]
        sns.histplot(depth, bins=bins_crse, stat='count',   element='step', linewidth=2, fill=False, label=proj, ax=ax)
    sns.despine(ax=ax)
    ax.grid(False)
    ax.set_xlabel('pia->L4')
    ax.legend()

    ax = axs.flat[1]
    for proj in projs:
        depth = res[sample_anno][proj]
        sns.histplot(depth, bins=bins_crse, stat='percent', element='step', linewidth=2, fill=False, label=proj, ax=ax)
    sns.despine(ax=ax)
    ax.grid(False)
    ax.set_xlabel('pia->L4')
    ax.legend()

    ax = axs.flat[2]
    for proj in projs:
        depth = res[sample_anno][proj]
        sns.histplot(depth, bins=bins_fine, stat='count',   cumulative=True, element='step', linewidth=2, fill=False, label=proj, ax=ax)
    ax.set_ylabel('Cumu. count')
    sns.despine(ax=ax)
    ax.grid(False)
    ax.set_xlabel('pia->L4')
    ax.legend()

    ax = axs.flat[3]
    for proj in projs:
        depth = res[sample_anno][proj]
        sns.histplot(depth, bins=bins_fine, stat='percent', cumulative=True, element='step', linewidth=2, fill=False, label=proj, ax=ax)
    sns.despine(ax=ax)
    ax.grid(False)
    ax.set_ylabel('Cumu. percent')
    ax.set_xlabel('pia->L4')
    ax.legend()

    plt.show()


In [ ]:
bins_fine = np.linspace(100, 512, num=64)
bins_crse = np.linspace(100, 512, num=64)

sample_annos = [
    'NR_adult',
    'DR_adult',
    'DR_P34',
    'DR_P34_v2',
    'NR_P14',
]
projs = ['LM', 'RL']

for sample_anno in sample_annos:
    title = sample_anno + ' ' + anno2samp[sample_anno][4:]
    plot(sample_anno, projs, bins_fine, bins_crse, title=title)
